# DP1 delivered image quality

Taken from SITCOMTN-149 and applied to DP1 data with pulication customisations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF

from lsst.daf.butler import Butler
import lsst.geom

# Set a standard figure size to use
from lsst.utils.plotting import publication_plots
from lsst.utils.plotting import get_multiband_plot_colors, get_multiband_plot_symbols, get_multiband_plot_linestyles

In [ ]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/raw/all', 
               'LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260' ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)

In [ ]:
# Applying the Rubin DP1 paper custom style sheet to all of the graphs to be created in this notebook
publication_plots.set_rubin_plotstyle()
bands_dict = publication_plots.get_band_dicts()
colors = get_multiband_plot_colors()
bands = colors.keys()  # important to get the right order for plot legends
%matplotlib inline

In [ ]:
visit_detector_table = butler.get("visit_detector_table", storageClass="DataFrame")

In [ ]:
# Extract and format data for analysis
df = visit_detector_table[['detectorId','visitId', 'band', 'psfSigma','seeing']].copy()
df["psfFwhm"]=df["psfSigma"]*2.355*0.2
df

In [ ]:
# Best delivered IQ
min_row = df.loc[df['psfFwhm'].idxmin()]
min_row

In [ ]:
# This is very low -- 0.135 arcsec seeing is not physical.
# Should these and other such visits be excluded from the dataset?
rows = df[df['visitId'] == 2024112400119]
rows

In [ ]:
# Compute and export out the IQ table percentile data
percentiles = [25, 50, 75]
iq_pctls = df.groupby('band')['psfFwhm'].quantile([p / 100 for p in percentiles]).apply(lambda x: round(x, 2)).unstack()

# Sort according to standard band order
iq_pctls.index = pd.Categorical(iq_pctls.index, categories=bands, ordered=True)
iq_pctls

In [ ]:
# Write to table for inclusion in latex table 
iq_pctls.to_csv('../tables/iq_data.csv', index=False)

In [ ]:
# Extract data from visit table for plotting
data = {}
for band in bands:
    data[band] = np.array(df[df['band'].str.contains(band)]['psfFwhm']) 

In [ ]:
# IQ histogram - not for inclusion in the paper but interesting
plt.figure()

for label, d in data.items():
    plt.hist(d, bins=20, alpha=0.5,
             linestyle='-',
             color = colors[label],
             label=label)

# Customize plot
plt.xlabel('PSF FWHM [arcsec]')
plt.ylabel('Fraction of Sensors')
plt.grid(True)
plt.legend()
# plt.xlim(0, 0.9)  
# plt.ylim(0, 500) 

plt.savefig("../figures/image_quality_histo.png", 
            bbox_inches='tight',  # Trim whitespace around the figure
            transparent=True,     # Transparent background
            format='png')         # Explicit format specification
plt.show()

In [ ]:
# IQ ECDF --  long tail in the i-band
plt.figure()

for label, d in data.items():
    ecdf = ECDF(d)
    plt.plot(ecdf.x, ecdf.y, 
             linestyle='-',
             color = colors[label],
             label=label)

# Customize plot
plt.xlabel('PSF FWHM [arcsec]')
plt.ylabel('Fraction of Sensors')
plt.grid(True)
plt.legend()
plt.savefig("../figures/image_quality_histo.png", 
            bbox_inches='tight',  # Trim whitespace around the figure
            transparent=True,     # Transparent background
            format='png')         # Explicit format specification
plt.show()

In [ ]:
# Investigate the variation by field -- expect to be minimal
# TODO

In [ ]:
plt.close()